# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bilibino,RU,68.0546,166.4372,25.16,68,74,5.28,broken clouds
1,1,Port Alfred,ZA,-33.5906,26.8910,60.53,85,0,8.70,clear sky
2,2,Khatanga,RU,71.9667,102.5000,29.23,97,99,5.86,light snow
3,3,Port Elizabeth,ZA,-33.9180,25.5701,58.32,90,5,4.00,clear sky
4,4,Padang,ID,-0.9492,100.3543,80.65,84,100,7.07,overcast clouds
5,5,Bredasdorp,ZA,-34.5322,20.0403,46.80,86,64,5.14,broken clouds
6,6,Busselton,AU,-33.6500,115.3333,57.49,61,75,18.81,broken clouds
7,7,Ahipara,NZ,-35.1667,173.1667,57.56,82,100,15.52,overcast clouds
8,8,Clovis,US,36.8252,-119.7029,79.84,46,0,5.75,clear sky
9,9,Banda Aceh,ID,5.5577,95.3222,79.70,82,100,7.83,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                   
# Display sample data
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Padang,ID,-0.9492,100.3543,80.65,84,100,7.07,overcast clouds
8,8,Clovis,US,36.8252,-119.7029,79.84,46,0,5.75,clear sky
9,9,Banda Aceh,ID,5.5577,95.3222,79.70,82,100,7.83,overcast clouds
21,21,Sao Filipe,CV,14.8961,-24.4956,78.06,78,82,0.29,broken clouds
30,30,Tual,ID,-5.6667,132.7500,83.25,73,94,11.01,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
622,622,Port Augusta,AU,-32.5000,137.7667,79.56,32,12,12.64,few clouds
638,638,Sola,VU,-13.8833,167.5500,81.18,76,23,17.22,few clouds
648,648,Black River,JM,18.0264,-77.8487,79.11,85,99,1.54,overcast clouds
649,649,Porto Velho,BR,-8.7619,-63.9039,80.65,78,0,6.91,clear sky


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Padang,ID,-0.9492,100.3543,80.65,84,100,7.07,overcast clouds
8,8,Clovis,US,36.8252,-119.7029,79.84,46,0,5.75,clear sky
9,9,Banda Aceh,ID,5.5577,95.3222,79.70,82,100,7.83,overcast clouds
21,21,Sao Filipe,CV,14.8961,-24.4956,78.06,78,82,0.29,broken clouds
30,30,Tual,ID,-5.6667,132.7500,83.25,73,94,11.01,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
622,622,Port Augusta,AU,-32.5000,137.7667,79.56,32,12,12.64,few clouds
638,638,Sola,VU,-13.8833,167.5500,81.18,76,23,17.22,few clouds
648,648,Black River,JM,18.0264,-77.8487,79.11,85,99,1.54,overcast clouds
649,649,Porto Velho,BR,-8.7619,-63.9039,80.65,78,0,6.91,clear sky


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [6]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City","Country","Max Temp","Current Description","Lat","Lng"]].copy()

# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng
4,Padang,ID,80.65,overcast clouds,-0.9492,100.3543
8,Clovis,US,79.84,clear sky,36.8252,-119.7029
9,Banda Aceh,ID,79.70,overcast clouds,5.5577,95.3222
21,Sao Filipe,CV,78.06,broken clouds,14.8961,-24.4956
30,Tual,ID,83.25,overcast clouds,-5.6667,132.7500
...,...,...,...,...,...,...
622,Port Augusta,AU,79.56,few clouds,-32.5000,137.7667
638,Sola,VU,81.18,few clouds,-13.8833,167.5500
648,Black River,JM,79.11,overcast clouds,18.0264,-77.8487
649,Porto Velho,BR,80.65,clear sky,-8.7619,-63.9039


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [7]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Padang,ID,80.65,overcast clouds,-0.9492,100.3543,
8,Clovis,US,79.84,clear sky,36.8252,-119.7029,
9,Banda Aceh,ID,79.70,overcast clouds,5.5577,95.3222,
21,Sao Filipe,CV,78.06,broken clouds,14.8961,-24.4956,
30,Tual,ID,83.25,overcast clouds,-5.6667,132.7500,
...,...,...,...,...,...,...,...
622,Port Augusta,AU,79.56,few clouds,-32.5000,137.7667,
638,Sola,VU,81.18,few clouds,-13.8833,167.5500,
648,Black River,JM,79.11,overcast clouds,18.0264,-77.8487,
649,Porto Velho,BR,80.65,clear sky,-8.7619,-63.9039,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [8]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [9]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Padang,ID,80.65,overcast clouds,-0.9492,100.3543,Mercure Padang
8,Clovis,US,79.84,clear sky,36.8252,-119.7029,Fairfield Inn & Suites by Marriott Fresno Clovis
9,Banda Aceh,ID,79.70,overcast clouds,5.5577,95.3222,OYO 884 Rumoh PMI Hotel
21,Sao Filipe,CV,78.06,broken clouds,14.8961,-24.4956,Hotel Xaguate
30,Tual,ID,83.25,overcast clouds,-5.6667,132.7500,Grand Vilia Hotel
...,...,...,...,...,...,...,...
622,Port Augusta,AU,79.56,few clouds,-32.5000,137.7667,Majestic Oasis Apartments
638,Sola,VU,81.18,few clouds,-13.8833,167.5500,Leumerus Bungalows
648,Black River,JM,79.11,overcast clouds,18.0264,-77.8487,Idlers' Rest Beach Hotel
649,Porto Velho,BR,80.65,clear sky,-8.7619,-63.9039,Ecos Hotel Conforto


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [11]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Padang,ID,80.65,overcast clouds,-0.9492,100.3543,Mercure Padang
8,Clovis,US,79.84,clear sky,36.8252,-119.7029,Fairfield Inn & Suites by Marriott Fresno Clovis
9,Banda Aceh,ID,79.70,overcast clouds,5.5577,95.3222,OYO 884 Rumoh PMI Hotel
21,Sao Filipe,CV,78.06,broken clouds,14.8961,-24.4956,Hotel Xaguate
30,Tual,ID,83.25,overcast clouds,-5.6667,132.7500,Grand Vilia Hotel
...,...,...,...,...,...,...,...
622,Port Augusta,AU,79.56,few clouds,-32.5000,137.7667,Majestic Oasis Apartments
638,Sola,VU,81.18,few clouds,-13.8833,167.5500,Leumerus Bungalows
648,Black River,JM,79.11,overcast clouds,18.0264,-77.8487,Idlers' Rest Beach Hotel
649,Porto Velho,BR,80.65,clear sky,-8.7619,-63.9039,Ecos Hotel Conforto


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [12]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [13]:
# Set the file name.
output_data_file = "WeatherPy_Vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [14]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [15]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Mercure Padang</dd>\n<dt>City</dt><dd>Padang</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 80.65 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Fairfield Inn & Suites by Marriott Fresno Clovis</dd>\n<dt>City</dt><dd>Clovis</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>clear sky and 79.84 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>OYO 884 Rumoh PMI Hotel</dd>\n<dt>City</dt><dd>Banda Aceh</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 79.7 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Xaguate</dd>\n<dt>City</dt><dd>Sao Filipe</dd>\n<dt>Country</dt><dd>CV</dd>\n<dt>Current Weather</dt><dd>broken clouds and 78.06 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Grand Vilia Hotel</dd>\n<dt>City</dt><dd>Tual</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 83.25 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [16]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
4,-0.9492,100.3543
8,36.8252,-119.7029
9,5.5577,95.3222
21,14.8961,-24.4956
30,-5.6667,132.7500
32,27.4833,78.3167
40,10.5835,-61.1177
49,28.2333,81.3333
52,3.0707,172.7902
57,25.3833,97.4000


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [17]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))